Before starting, converting data for regions


data = pd.io.stata.read_stata('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/wm_replication_dataset_2020_09_25.dta')
data.to_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/region_info.csv')

In [530]:
import pandas as pd
import googletrans
from googletrans import Translator
import time


Import Scraped Data on City Politics from other module

In [531]:
#politics = 

Importing csv's of regions and femicides cases

In [532]:
regions_info = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/region_info.csv')

In [533]:
victims = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/victim_level_data.csv')

# Data Cleaning
Starting with the regions.
Columns data:
- Province Code
- Province Name
- all_wm_prov: women’s murders in Turkey (per 100,000 people) per year
- knows_wm_prov: number of women’s murders (again, per 100,000 people) across Turkish
provinces
- tot_pop
- ethnic_turkish
- ethnic_kurdish
- ln_gdppc_cons: e (logged) GDP per capita 
- fm_diff_educ: Gender equality in education attainment is calculated by subtracting the percentage of men who finished at least high school from the percentage of women who finished at least high school
- divore_rate: every observation, the average divorce
rate (i.e. number of divorces per 1000 people) in the last five years is calculated
- mosques: Religiosity is measured by the number of mosques (per 1000 people) in a province
- ln_casualty: civil war is measured by the (logged) number of war-related killings that occurred in a
province between 1984 and 2010
- ceasefire: It takes the value of 1 in years 2013 and 2014, and 0 otherwise. This variable is interacted with Civil War Exposure.

In [534]:
regions_info = regions_info.drop('Unnamed: 0', axis=1)

In [535]:
regions_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   prov_code         2460 non-null   int64  
 1   prov_name         2460 non-null   object 
 2   year              2460 non-null   int64  
 3   all_wm_prov       648 non-null    float64
 4   knows_wm_prov     648 non-null    float64
 5   total_pop         648 non-null    float64
 6   ethnic_turkish    648 non-null    float64
 7   ethnic_kurdish    648 non-null    float64
 8   ln_casualty       648 non-null    float64
 9   ln_gdppc_cons     891 non-null    float64
 10  divorce_rate_ma5  1377 non-null   float64
 11  mosques_perk      648 non-null    float64
 12  ceasefire         2460 non-null   int64  
 13  fm_diff_educ      810 non-null    float64
dtypes: float64(10), int64(3), object(1)
memory usage: 269.2+ KB


We can already drop the years we do not need. Our dataset on femicides goes from 2010 to 2017. So for now, I will only keep data from 2009 to 2017.

In [536]:
regions_info.drop(regions_info[regions_info['year'] < 2010].index, inplace = True) 

In [537]:
regions_info.drop(regions_info[regions_info['year'] > 2017].index, inplace = True) 

In [538]:
regions_info['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype=int64)

We do not convert to datetime as we only have the years.

## Exploration of data regions

Check for duplicates

In [539]:
sum(regions_info.duplicated())

0

Check for nulls. Nothing needed.

In [540]:
round(regions_info.isna().sum()/len(regions_info),4)*100

prov_code           0.0
prov_name           0.0
year                0.0
all_wm_prov         0.0
knows_wm_prov       0.0
total_pop           0.0
ethnic_turkish      0.0
ethnic_kurdish      0.0
ln_casualty         0.0
ln_gdppc_cons       0.0
divorce_rate_ma5    0.0
mosques_perk        0.0
ceasefire           0.0
fm_diff_educ        0.0
dtype: float64

Check data details

In [541]:
regions_info.describe(include = 'all')

,prov_code,prov_name,year,all_wm_prov,knows_wm_prov,total_pop,ethnic_turkish,ethnic_kurdish,ln_casualty,ln_gdppc_cons,divorce_rate_ma5,mosques_perk,ceasefire,fm_diff_educ
count,648.000000,648,648.000000,648.000000,648.000000,6.480000e+02,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000
unique,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,MERSIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,41.000000,NaN,2013.500000,3.030864,2.915123,9.534026e+05,75.425493,21.467349,2.345763,4.344408,1.283133,1.843475,0.250000,-0.100884
std,23.398966,NaN,2.293058,5.068218,4.868333,1.706082e+06,33.867957,32.434790,2.623615,0.342109,0.616719,1.160641,0.433347,0.028900
min,1.000000,NaN,2010.000000,0.000000,0.000000,7.441200e+04,0.000000,0.000000,0.000000,3.463987,0.114000,0.226302,0.000000,-0.184529
25%,21.000000,NaN,2011.750000,0.000000,0.000000,2.808055e+05,62.074455,0.000000,0.000000,4.108124,0.857000,1.120346,0.000000,-0.119214
50%,41.000000,NaN,2013.500000,1.000000,1.000000,5.131070e+05,94.709680,3.169473,1.098612,4.336241,1.342000,1.577232,0.000000,-0.099548
75%,61.000000,NaN,2015.250000,4.000000,3.000000,9.948722e+05,99.407780,25.884853,4.204693,4.566989,1.709000,2.132194,0.250000,-0.080654


We drop the province name and only keep the province code

In [562]:
regions_info = regions_info.drop('prov_name', axis=1)

# Data Cleaning and Translations for Victims Cases

In [542]:
victims.rename(columns = {'isim':'victim_name', 'yaş':'age', 'tarih':'date',
                         'yer':'place','fail':'perpetrator',
                         'bahane':'excuse', 'hikaye':'story' }, inplace = True) 


In [543]:
victims1 = victims.drop('link', axis=1)

In [544]:
victims1 = victims1.drop('victim_name', axis=1)

Splitting locations to have a better overview on cities

In [545]:
victims2 = victims1.join(victims['place'].str.split(',', 1, expand=True).rename(columns={0:'district', 1:'city'}))


In [546]:
victims2 = victims2.drop('place', axis=1)

Translate from Turkish 

In [547]:
victims2['perpetrator'].unique()

array(['Erkek arkadaşı', 'Kocası', 'Oğlu', 'Eski kocası',
       'Tanıdığı bir erkek', 'Babası', 'Diğer yakını',
       'Dini nikahlı kocası', 'Üvey oğlu', 'Erkek kardeşi',
       'Eski erkek arkadaşı', 'Akrabası', 'Damadı', 'Saldırgan',
       'Dini nikahlı eski kocası', 'Kayınpederi'], dtype=object)

In [548]:
translations = {
        'Erkek arkadaşı':'boyfriend',
        'Kocası':'husband',
        'Oğlu':'son',
        'Eski kocası':'ex-husband',
        'Tanıdığı bir erkek':'known man',
        'Babası':'father',
        'Diğer yakını':'other relative',
        'Dini nikahlı kocası':'religiously-married husband',
        'Üvey oğlu':'step-son',
        'Erkek kardeşi':'brother',
        'Eski erkek arkadaşı':'ex-boyfriend',
        'Akrabası':'relative',
        'Damadı':'fiancè',
        'Saldırgan':'attacker', 
        'Dini nikahlı eski kocası':'religiously married ex-husband', 
        'Kayınpederi':'father-in-law'
    }

victims2['perpetrator_en'] = victims2['perpetrator'].replace(translations)

Remapping perpetratore categories

In [549]:
remapping_perpetrator = {
        'boyfriend': ['boyfriend'],
        'partner':['fiancè','husband','religiously-married husband'],
        'son':['son', 'step-son'],
        'known man': ['known man'],
        'father':['father'],
        'relative':['other relative','relative'],
        'brother':['brother'],
        'attacker':['attacker'],
        'ex-partner':['ex-husband','religiously married ex-husband', 'ex-boyfriend'],
        'father-in-law':['father-in-law']
    }
victims2['perpetrator_en_new'] = victims2['perpetrator_en'].map(lambda x: next((k for k, v in remapping_perpetrator.items() if x in v), None))

In [550]:
victims2 = victims2.drop('perpetrator', axis=1)


In [551]:
victims2 =victims2.drop('perpetrator_en', axis=1)

In [552]:
victims2.rename(columns={'perpetrator_en_new': 'perp_cat'})

,age,date,excuse,story,district,city,perp_cat
0,18.0,31/12/2017,Bilinmiyor,"Erhan M. adlı kişi, sevgilisi Buket Şişek'i 5....",Esenyurt,Istanbul,boyfriend
1,17.0,31/12/2017,Kadının ayrılma isteği,"19 yaşındaki Furkan K., kendisinden ayrılmak i...",Ağlasun,Burdur,boyfriend
2,38.0,30/12/2017,Tartışma/ kavga,"Tevfik K. adlı kişi, tartışma üzerine sevgilis...",Merkez,Bolu,boyfriend
3,53.0,27/12/2017,Kadının ayrılma isteği,"Seyfali Akkoç adlı kişi, bir süredir kendisind...",Etimesgut,Ankara,boyfriend
4,22.0,25/12/2017,Aldatılma şüphesi,Şeyhmus İ. adlı kişi kendisini aldattığı iddia...,Yenişehir,Diyarbakır,partner
...,...,...,...,...,...,...,...
1959,24.0,14/01/2010,Erkeğin ayrılma isteğinin reddedilmesi,"İddiaya göre Yunus F., eşinin kardeşinin karıs...",Mamak,Ankara,relative
1960,34.0,12/1/2010,Erkeğin reddedilmesi/ terk edilmesi,Cezaevindeki iyi hali nedeniyle Adana Adliyesi...,Seyhan,Adana,attacker
1961,57.0,12/1/2010,Bilinmiyor,"48 yaşındaki Yusuf D., evine gelen ablası Alim...",Merkez,Kütahya,brother
1962,34.0,11/1/2010,Bilinmiyor,"26 yaşındaki Uzman Onbaşı Harun Ceylan, sevgil...",Ceyhan,Adana,boyfriend


Decided to drop the stories altogether

In [553]:
victims2= victims2.drop('story', axis=1)

Translating the excuse

In [554]:
#victims2['excuse'].unique()

In [555]:
#from google_trans_new import google_translator  
#translator = google_translator()  
#translations = {}
#for column in victims2.columns:
    # unique elements of the column
 #   unique_elements = victims2[column].unique()
  #  for element in unique_elements:
        # add translation to the dictionary
         #time.sleep(10)
      #  translations[element] = translator.translate(element)
    
#print(translations[5])

Getting a too many requests error from google translate API, translating the excuse with a simple map

In [556]:
translations_excuses= {
        'Bilinmiyor':'Unknown',
'Kadının ayrılma isteği':'The womans desire to leave',
'Tartışma/ kavga':' Argument / fight',
'Aldatılma şüphesi':'Suspicion of being deceived',
'Erkeğin barışma isteğinin reddedilmesi':'Refusing the mans request for reconciliation',
'Maddi kaynaklı':'Financially sourced',
'Diğer, Erkeğin "erkekliği"':'Other, the man s manhood',
'Diğer, Psikolojik sorun':'Other Psychological problem',
'Erkeğin reddedilmesi/ terk edilmesi':'The rejection / abandonment of the man',
'Kaza iddiası':'Accident claim',
'Dolaylı':'Indirect',
'Kadının karar ve tercihlerine ilişkin, Kadının seks işçiliğine karşı direnmesi':'Regarding the decisions and preferences of women womens resistance to sex work',
'Kadının boşanma isteği':'The womans desire for divorce',
'Namus/ töre':'Honor',
'Kadının karar ve tercihlerine ilişkin, Kadının dedikodu yapması':'The womans gossip about the womans decisions and preferences',
'Kıskançlık':'Jealousy',
'Cinsel saldırı':'Sexual assault',
'Diğer, Başka bir olaydan kadını sorumlu tutma':'Other holding a woman responsible for another incident',
'Kadının karar ve tercihlerine ilişkin, Kadının hakaret etmesi':'Regarding the womans decisions and preferences the womans insulting',
'Kadının karar ve tercihlerine ilişkin, Kadının erkekten içki içmemesini istemesi':'Regarding the womans decisions and preferences the womans asking the man not to drink alcohol',
'Kadının karar ve tercihlerine ilişkin, Kadının sosyal medya hesabı':'Womens social media account regarding the decisions and preferences of the woman',
'Diğer, Yönlendirme':'Other Routing',
'Çocuk/ velayet kaynaklı':'Child / custody sourced',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin ilişkisini onaylamaması':'Regarding the womans decisions and preferences the womans disapproval of the mans relationship',
'Kadının karar ve tercihlerine ilişkin, Kadının evden ayrılmak istemesi':'Regarding the womans decisions and preferences the womans willingness to leave the house',
'Hırsızlık':'Theft',
'Erkeğin hizmet beklentisi, Kadının yemeğe salça koyması':'Male service expectation Woman putting tomato paste in food',
'Erkeğin ayrılma isteğinin reddedilmesi':'Refusal of a mans request to leave',
'Kadının karar ve tercihlerine ilişkin, Erkeğin kadının işini onaylamaması':'Regarding the womans decisions and preferences the mans disapproval of the womans job',
'Kadının karar ve tercihlerine ilişkin, Erkeğin kadının ilişkisini onaylamaması':'Regarding the womans decisions and preferences the mans disapproval of the womans relationship',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin tacizini ifşa etmesi':'Regarding the decisions and preferences of the woman the womans disclosure of the harassment of the man',
'Diğer, Erkeğe vahiy gelmesi':'The other is the coming of revelation to the man',
'Diğer, Erkeğe cinayeti cinlerin söylemesi':'Other the murder of the man is told by the jinn',
'Cinsiyet kimliği':'Gender identity',
'Kadının karar ve tercihlerine ilişkin, Kadının telefon şifresini söylememesi':'Regarding the womans decisions and preferences the womans not saying her phone password',
'Kadının karar ve tercihlerine ilişkin, Kadının abisinde kalması':'Regarding the decisions and preferences of the woman the woman should remain in her brother',
'Diğer, Erkeğin sinirlerine hakim olamaması':'The other is that the man cannot control his nerves',
'Diğer, Erkeğin uyuşturucunun etkisine girmesi':'The other is that the man is under the influence of drugs',
'Kadının karar ve tercihlerine ilişkin, Kadının ailesinde kalmak istemesi':'Regarding the womans decisions and preferences the womans willingness to stay in her family',
'Diğer, Erkeğin cinayetle suçlanması':'The other is that the man is accused of murder',
'Kadının karar ve tercihlerine ilişkin, Kadının kardeşini koruması':'Regarding the womans decisions and preferences Womans protection of her brother',
'Diğer, Kadının uykusunda erkeğe saldırması':'Other a woman attacking a man in her sleep',
'Kadının karar ve tercihlerine ilişkin, Kadının telefonda çok konuşması':'Women talking a lot on the phone regarding the decisions and preferences of the woman',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeği şikayet etmesi':'Regarding the womans decisions and preferences Womans complaint against the man',
'Kadının karar ve tercihlerine ilişkin, Kadının boşandığı eşine dönmesi':'Regarding the womans decisions and preferences the womans return to her divorced spouse',
'Kadının karar ve tercihlerine ilişkin, Kadının sık sevgili değiştirmesi':'Regarding the womans decisions and preferences the womans frequent change of lovers',
'Kadının karar ve tercihlerine ilişkin, Kadının geç saatlere kadar gezmesi':'Regarding the womans decisions and preferences Womans traveling until late hours',
'Diğer, Erkeğin bunalması':'Other the man is overwhelmed',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeği rezil edeceğini söylemesi':'Regarding the womans decisions and preferences the womans saying that she will disgrace the man',
'Diğer, Erkeğin gürültüden rahatsız olması':'The other is that the man is disturbed by the noise',
'Kadının karar ve tercihlerine ilişkin, Kadının yurttan kaçması':'Regarding the womans decisions and preferences Womans escape from the dormitory',
'Diğer, Erkeğin söylediği yalanın ortaya çıkması':'The other is the disclosure of the lie that the man told',
'Erkeğin hizmet beklentisi, Kadının su vermemesi':'Male service expectation Woman not giving water',
'Kadının karar ve tercihlerine ilişkin, Kadının kızına aldığı elbisenin etek boyu':'Regarding the decisions and preferences of the woman the skirt length of the dress the woman bought for her daughter',
'Diğer, Kadının telefonuna mesaj gelmesi':'Other a message on the womans phone',
'Erkeğin hizmet beklentisi, Kadının erkeğin derdini dinlememesi':'Male service expectation Woman not listening to mans troubles',
'Diğer, Cinsiyet':'Other Gender',
'Erkeğin hizmet beklentisi, Kadının yemek hazırlamaması':'Male service expectation Woman not preparing food',
'Kadının karar ve tercihlerine ilişkin, Kadının saçını kızıla boyatması':'Regarding the womans decisions and preferences the woman dying her hair red',
'Kadının karar ve tercihlerine ilişkin, Kadının yaptırdığı dövme':'A tattoo made by the woman regarding the decisions and preferences of the woman',
'Erkeğin hizmet beklentisi, Kadının çamaşır yıkamaması':'Male service expectation Woman not washing clothes',
'Erkeğin hizmet beklentisi, Kadının erkeğin annesini seyrek ziyaret etmesi':'Male service expectation Womans frequent visits to her mother',
'Kadının karar ve tercihlerine ilişkin, Kadının gülmesi':'Regarding the decisions and preferences of the woman the laugh of the woman',
'Kadının karar ve tercihlerine ilişkin, Kadının ülkesine dönme isteği':'Regarding the decisions and preferences of the woman the desire of the woman to return to her country',
'Diğer, Erkeğin kuma getirme isteğinin reddedilmesi':'Other refusal of the mans request to bring it to the sand',
'Diğer, Erkeğe evliliğin zor gelmesi':'Other it is difficult to get married to a man',
'Diğer, Kadının evlenme isteği':'Other the womans desire to marry',
'Kadının karar ve tercihlerine ilişkin, Kadının akrabalarıyla görüşmesi':'The womans meeting with her relatives regarding the womans decisions and preferences',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin kardeşine yardım etmesi':'Regarding the womans decisions and preferences the woman helping the mans brother',
'Erkeğin hizmet beklentisi, Kadının erkek çocuk doğurmaması':'Male service expectation Woman not giving birth to male children',
'Kadının karar ve tercihlerine ilişkin, Kadının nafaka talebi':'Regarding the womans decisions and preferences the womans alimony request',
'Erkeğin hizmet beklentisi, Kadının erkeğe ilgi göstermemesi':'Male service expectation Woman not showing interest in men',
'Kadının karar ve tercihlerine ilişkin, Kadının otomobil istemesi':'Regarding the womans decisions and preferences the womans request for a car',
'Kadının karar ve tercihlerine ilişkin, Kadının elbise istemesi':'Regarding the womans decisions and preferences the womans request for a dress',
'Kadının karar ve tercihlerine ilişkin, Kadının çocuğu dövmesi':'Regarding the womans decisions and preferences Womans tattooing the child',
'Diğer, Kan davası':'Other Blood feud',
'Diğer, Erkeğin gıcık olması':'The other is that the man is annoyed',
'Kadının karar ve tercihlerine ilişkin, Kadının aleyhte tanıklık yapması':'The womans testimony against the womans decisions and preferences',
'Kadının karar ve tercihlerine ilişkin, Kadının eve geç gelmesi':'Regarding the womans decisions and preferences Woman coming home late',
'Erkeğin hizmet beklentisi, Kadının patates köfte yapmaması':'Male service expectation Woman not making potato meatballs',
'Diğer, Sebepsiz':'Other No Reason',
'Kadının karar ve tercihlerine ilişkin, Kadının küsmesi':'Regarding the decisions and preferences of the woman Woman being offended',
'Kadının karar ve tercihlerine ilişkin, Kadının yazlık ev istemesi':'Regarding the womans decisions and preferences the womans request for a summer house',
'Erkeğin hizmet beklentisi, Kadının çocuk sahibi olmaması':'Male service expectation Woman not having children',
'Erkeğin hizmet beklentisi, Kadının erkeğe iyi bakmaması':'Male service expectation Woman not taking good care of men',
'Erkeğin hizmet beklentisi, Kadının evi kirletmesi':'Male service expectation Woman polluting the house',
'Erkeğin boşanma isteğinin reddedilmesi':'The rejection of a mans divorce request',
'Kadının karar ve tercihlerine ilişkin, Kadının aracı yanlış yönlendirmesi':'Woman misleads the tool regarding the decisions and preferences of the woman',
'Diğer, Çocuğun yemek yememesi':'Other If the child does not eat',
'Erkeğin hizmet beklentisi, Kadının tuzluğu uzatmaması':'Male service expectation Woman not extending the salt shaker',
'Erkeğin kadının işini onaylamaması':'The man does not approve of the womans job',
'Erkeğin hizmet beklentisi, Kadının yemeği geç hazırlaması':'Male service expectation Woman preparing dinner late',
'Erkeğin hizmet beklentisi, Kadının ev işlerini yapmaması':'Male service expectation Woman not doing housework',
'Erkeğin hizmet beklentisi, Kadının sabah erkeği uyandırmaması':'Male service expectation Woman not waking up the man in the morning',
'Kadının karar ve tercihlerine ilişkin, Kadının yeni elbise alması':'Regarding the womans decisions and preferences the womans buying new clothes',
'Kadının karar ve tercihlerine ilişkin, Kadının başörtüsü takmak istememesi':'Regarding the womans decisions and preferences the woman does not want to wear a headscarf',
'Erkeğin hizmet beklentisi, Kadının koyunları otlatmaması':'Male service expectation woman not grazing sheep',
'Kadının karar ve tercihlerine ilişkin, Kadının oğlunda kalma isteği':'Regarding the womans decisions and preferences the womans desire to stay with her son',
 }

victims2['excuse_en'] = victims2['excuse'].replace(translations_excuses)

In [557]:
len(victims2['excuse_en'].unique())

98

# Remapping
After translations, I have realized there are too many categories for the excuses given for the femicides. I will cluster them in fewer buckets.

The main categories of excuses seem to be:
- Unknown
- Psychological issue
- Money Problems
- Woman quest for freedom (i.e. leaving the household, asking for divorce, etc.)
- Indirect
- Household expectations (i.e. not waking up the man, preparing dinner late, 
- Body expectations (i.e. not having children, 

In [558]:
remapping_excuses = {
        'Children':['Child / custody sourced',
                    'Other If the child does not eat',
                    'Regarding the womans decisions and preferences the womans desire to stay with her son'],
        'Fight':['Other holding a woman responsible for another incident'],
        'Household service expectations':['Male service expectation Woman putting tomato paste in food',
                                          'Male service expectation Woman not giving water',
                                          'Male service expectation Woman not preparing food',
                                          'Male service expectation',
                                          'Woman not washing clothes',
                                          'Male service expectation Woman not extending the salt shaker',
                                          'Male service expectation Woman preparing dinner late',
                                          'Male service expectation Woman not doing housework',
                                          'Male service expectation Woman not waking up the man in the morning',
                                         'Male service expectation woman not grazing sheep',
                                         'Male service expectation Woman not making potato meatballs',
                                         'Male service expectation Woman not washing clothes'],
        'Male service expectations':['Male service expectation Woman not listening to mans troubles',
                                     'Male service expectation Womans frequent visits to her mother',
                                     'Male service expectation Woman not giving birth to male children',
                                     'Male service expectation Woman not showing interest in men',
                                     'Male service expectation Woman not having children',
                                     'Male service expectation Woman polluting the house',
                                     'The man does not approve of the womans job',
                                    'The other is that the man is annoyed'],
        'Masculinity&Honor':['Other, the man s manhood',
                             'Honor',
                             'Jealousy',
                             'Refusal of a mans request to leave',
                             'Other the man is overwhelmed',
                             'Other a message on the womans phone',
                             'Male service expectation Woman not taking good care of men',
                            'The other is that the man cannot control his nerves'],
        'Other':[' Argument / fight','Financially sourced',
                 'Other Psychological problem',
                 'Accident claim',
                 'Other Routing',
                 'The other is the coming of revelation to the man',
                 'The other is that the man is under the influence of drugs',
                 'The other is that the man is accused of murder',
                 'The other is that the man is disturbed by the noise',
                 'The other is the disclosure of the lie that the man told',
                'Other Gender',
                 'Other refusal of the mans request to bring it to the sand',
                 'Other it is difficult to get married to a man',
                 'Other Blood feud',
                 'Other No Reason','Theft','Indirect'],
        'Separation & Jealousy':['Refusing the mans request for reconciliation',
                                 'The womans desire to leave',
                                 'Suspicion of being deceived',
                                 'The rejection / abandonment of the man',
                                 'Regarding the womans decisions and preferences the womans disapproval of the mans relationship',
                                 'Regarding the womans decisions and preferences the womans return to her divorced spouse',
                                 'The rejection of a mans divorce request',
                                'The womans desire for divorce',
                                'Regarding the womans decisions and preferences the womans willingness to leave the house'],
        'Sexual expectation':['Regarding the decisions and preferences of women womens resistance to sex work',
                              'Sexual assault',
                              'Other a woman attacking a man in her sleep'],
        'Unknown':['Unknown'],
        'Woman Decision and Preferences':['The womans gossip about the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences the womans insulting',
                                        'Regarding the womans decisions and preferences the womans asking the man not to drink alcohol',
                                        'Womens social media account regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences the mans disapproval of the womans job',
                                        'Regarding the womans decisions and preferences the mans disapproval of the womans relationship',
                                        'Regarding the decisions and preferences of the woman the womans disclosure of the harassment of the man',
                                        'Other the murder of the man is told by the jinn',
                                        'Gender identity',
                                        'Regarding the womans decisions and preferences the womans not saying her phone password',
                                        'Regarding the decisions and preferences of the woman the woman should remain in her brother',
                                        'Regarding the womans decisions and preferences the womans willingness to stay in her family',
                                        'Regarding the womans decisions and preferences Womans protection of her brother',
                                        'Women talking a lot on the phone regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences Womans complaint against the man',
                                        'Regarding the womans decisions and preferences the womans frequent change of lovers',
                                        'Regarding the womans decisions and preferences Womans traveling until late hours',
                                        'Regarding the womans decisions and preferences the womans saying that she will disgrace the man',
                                        'Regarding the womans decisions and preferences Womans escape from the dormitory',
                                        'Regarding the decisions and preferences of the woman the skirt length of the dress the woman bought for her daughter',
                                        'Regarding the womans decisions and preferences the woman dying her hair red',
                                        'A tattoo made by the woman regarding the decisions and preferences of the woman',
                                        'Regarding the decisions and preferences of the woman the laugh of the woman',
                                        'Regarding the decisions and preferences of the woman the desire of the woman to return to her country',
                                        'Other the womans desire to marry',
                                        'The womans meeting with her relatives regarding the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences the woman helping the mans brother',
                                        'Regarding the womans decisions and preferences the womans alimony request',
                                        'Regarding the womans decisions and preferences the womans request for a car',
                                        'Regarding the womans decisions and preferences the womans request for a dress',
                                        'Regarding the womans decisions and preferences Womans tattooing the child',
                                        'The womans testimony against the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences Woman coming home late',
                                        'Regarding the decisions and preferences of the woman Woman being offended',
                                        'Regarding the womans decisions and preferences the womans request for a summer house',
                                        'Woman misleads the tool regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences the womans buying new clothes',
                                        'Regarding the womans decisions and preferences the woman does not want to wear a headscarf']
    }
victims2['excuses_cat'] = victims2['excuse_en'].map(lambda x: next((k for k, v in remapping_excuses.items() if x in v), None))

Dropping old excuses column

In [559]:
victims2 = victims2.drop('excuse', axis=1)

In [560]:
victims2 = victims2.drop('excuse_en', axis=1)

In [561]:
victims2

,age,date,district,city,perpetrator_en_new,excuses_cat
0,18.0,31/12/2017,Esenyurt,Istanbul,boyfriend,Unknown
1,17.0,31/12/2017,Ağlasun,Burdur,boyfriend,Separation & Jealousy
2,38.0,30/12/2017,Merkez,Bolu,boyfriend,Other
3,53.0,27/12/2017,Etimesgut,Ankara,boyfriend,Separation & Jealousy
4,22.0,25/12/2017,Yenişehir,Diyarbakır,partner,Separation & Jealousy
...,...,...,...,...,...,...
1959,24.0,14/01/2010,Mamak,Ankara,relative,Masculinity&Honor
1960,34.0,12/1/2010,Seyhan,Adana,attacker,Separation & Jealousy
1961,57.0,12/1/2010,Merkez,Kütahya,brother,Unknown
1962,34.0,11/1/2010,Ceyhan,Adana,boyfriend,Unknown


## Export Cleaned Files

In [563]:
victims2.to_csv('victims_cleaned.csv')

In [564]:
regions_info .to_csv('regions_info.csv')